***GENERATED CODE FOR appointmentnoshowprediction PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.ml.feature import StringIndexer
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import round
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PatientId', 'threshold': 139932400316304.92, 'transformation_label': 'Binarizer'}], 'feature': 'PatientId', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
                                              'count': '500', 'mean': '139932400316304.92', 'stddev': '242520780616046.88', 'min': '2.51541539E8', 'max': '9.98287E14', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'PatientId'}, {'feature_label': 'PatientId', 'threshold': 139932400316304.92, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('PatientId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'ScheduledDay', 'transformation_label': 'Extract Date'}], 'feature': 'ScheduledDay', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'ScheduledDay'}, {'feature_label': 'ScheduledDay', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('ScheduledDay')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'AppointmentDay', 'transformation_label': 'Extract Date'}], 'feature': 'AppointmentDay', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'AppointmentDay'}, {'feature_label': 'AppointmentDay', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('AppointmentDay')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Neighbourhood', 'transformation_label': 'String Indexer'}], 'feature': 'Neighbourhood', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ANDORINHAS', 'max': 'VILA RUBIM', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Neighbourhood'}, {'feature_label': 'Neighbourhood', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Neighbourhood')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'No-show', 'transformation_label': 'String Indexer'}], 'feature': 'No-show', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'No-show'}, {'feature_label': 'No-show', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('No-show')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
#%run appointmentnoshowpredictionHooks.ipynb
try:
	#sourcePreExecutionHook()

	train = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/1614028143859_Train.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line', 'FilePath': '/Healthcare - RCM/Appointment No-Show Prediction/Train.csv', 'filename': '1614028143859_Train.csv', 'viewFileName': 'Train.csv'}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run appointmentnoshowpredictionHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(train,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "PatientId", "threshold": 139932400316304.92, "transformation_label": "Binarizer"}], "feature": "PatientId", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "139932400316304.92", "stddev": "242520780616046.88", "min": "2.51541539E8", "max": "9.98287E14", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "PatientId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AppointmentID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "5647685.38", "stddev": "70992.37", "min": "5284407", "max": "5754473", "missing": "0"}, "updatedLabel": "AppointmentID"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"feature_label": "ScheduledDay", "transformation_label": "Extract Date"}], "feature": "ScheduledDay", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "ScheduledDay"}, {"transformationsData": [{"feature_label": "AppointmentDay", "transformation_label": "Extract Date"}], "feature": "AppointmentDay", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "AppointmentDay"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "38.27", "stddev": "25.08", "min": "0", "max": "94", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Neighbourhood", "transformation_label": "String Indexer"}], "feature": "Neighbourhood", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ANDORINHAS", "max": "VILA RUBIM", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Neighbourhood"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Scholarship", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.08", "stddev": "0.27", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Scholarship"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Hipertension", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.2", "stddev": "0.4", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Hipertension"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Diabetes", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.07", "stddev": "0.26", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Diabetes"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Alcoholism", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.02", "stddev": "0.15", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Alcoholism"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Handcap", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.03", "stddev": "0.19", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Handcap"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SMS_received", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.3", "stddev": "0.46", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "SMS_received"}, {"transformationsData": [{"feature_label": "No-show", "transformation_label": "String Indexer"}], "feature": "No-show", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "No-show"}, {"feature": "PatientId_binarizer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "0.22", "stddev": "0.41", "min": "0.0", "max": "1.0", "missing": "0"}, "updatedLabel": "PatientId_binarizer_trans..."}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.43", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Gender_stringindexer_tran..."}, {"feature": "ScheduledDay_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "17.91", "stddev": "9.15", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "ScheduledDay_dayofmonth"}, {"feature": "ScheduledDay_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "4.46", "stddev": "0.7", "min": "2", "max": "6", "missing": "0"}, "updatedLabel": "ScheduledDay_month"}, {"feature": "ScheduledDay_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "2016.0", "stddev": "0.0", "min": "2016", "max": "2016", "missing": "0"}, "updatedLabel": "ScheduledDay_year"}, {"feature": "AppointmentDay_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "19.28", "stddev": "9.96", "min": "2", "max": "31", "missing": "0"}, "updatedLabel": "AppointmentDay_dayofmonth"}, {"feature": "AppointmentDay_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "5.0", "stddev": "0.0", "min": "5", "max": "5", "missing": "0"}, "updatedLabel": "AppointmentDay_month"}, {"feature": "AppointmentDay_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "2016.0", "stddev": "0.0", "min": "2016", "max": "2016", "missing": "0"}, "updatedLabel": "AppointmentDay_year"}, {"feature": "Neighbourhood_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "12.86", "stddev": "15.04", "min": "0.0", "max": "64.0", "missing": "0"}, "updatedLabel": "Neighbourhood_stringindex..."}, {"feature": "No-show_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.2", "stddev": "0.4", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "No-show_stringindexer_tra..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run appointmentnoshowpredictionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(autofe, ["PatientId_binarizer", "AppointmentID", "Gender_stringindexer", "Age", "Neighbourhood_stringindexer", "Scholarship", "Hipertension", "Diabetes", "Alcoholism", "Handcap", "SMS_received", "ScheduledDay_dayofmonth", "ScheduledDay_month", "ScheduledDay_year", "AppointmentDay_dayofmonth", "AppointmentDay_month", "AppointmentDay_year"], "No-show_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

